In [29]:

# 0 00001 0000101111
# 0 00010 0000101111
# 1. Take smaller, shift -> 0 00010 0000010111
# 2. Focus on mantissas. Add
#  1.0000010111 + 
#  1.0000101111 = 
# 10.0001000110
# Final:
# 01.0000100011


OPCODE = {
    "add": "0000",
    "lsl": "0001",
    "lsr": "0010",
    "and": "0011",
    "orr": "0100",
    "cmp": "0101",
    "not": "0110",
    "mov": "1001",
    "ldr": "1101", # special case XXXX_RD_LUT
    "str": "1110", # special case XXXX_RD_LUT
    "ina": "0111", # special case XXXX_SL_000
    "bne": "1111", # special case XXXX_BLUTTT
    "bge": "1010", # special case XXXX_BLUTTT
}

REG_LDR_STR = {
    "r0":"00",
    "r1":"01",
    "r2":"10",
    "r3":"11"
}

REG = {
    "r0":"000",
    "r1":"001",
    "r2":"010",
    "r3":"011",
    "r4":"100",
    "r5":"101",
    "r6":"110",
    "r7":"111",
}

def reg(reg):
   out = REG.get(reg)
   if out is None:
       raise ValueError(f"Register {reg} is not addressable.")
   return out

def process_line(line):
    insn = line.split()
    # print (insn)
    op = insn[0]
    opcode = OPCODE.get(op)
    # make sure opcode is valid
    if opcode is None: 
        raise ValueError(f"Operation {op} is not supported.")
    # Handle branch
    elif opcode == "1111" or opcode == "1010":
        return f"{opcode}{insn[1]}"
    # Handle load and store
    elif opcode == "1101" or opcode == "1110":
        rd = REG_LDR_STR.get(insn[1])
        if rd is None: raise ValueError(f"Register {rd} is not addressable.")
        return f"{opcode}{rd}{insn[2]}"
    # Handle ina
    elif opcode == "0111":
        return f"{opcode}{insn[1]}000"
    # Handle ALU operation
    else:
        rd = reg(insn[1])
        if opcode == "1001":
            print(REG_LDR_STR[insn[2]])
            return f"{opcode}{rd}{REG_LDR_STR[insn[2]]}"
        else:
            return f"{opcode}{rd}00"
        
    
    
annotation = False
in_multiline_comment = False

with open('pgm3update6.txt', 'r') as file, open('assembled_ms3_pgm3_v9.txt', 'w') as output_file:
    for line in file:
        line = line.strip()
        
        # Skip blank lines
        if not line:
            continue

        # Handle multi-line comments
        if '/*' in line:
            in_multiline_comment = True
            line = line.split('/*')[0].strip()  # Keep content before the '/*' if any
            
        if '*/' in line:
            in_multiline_comment = False
            line = line.split('*/')[-1].strip()  # Keep content after the '*/' if any

        # If inside a multi-line comment, skip this line
        if in_multiline_comment or not line:
            continue

        # Skip single-line comments
        if line.startswith('//'):
            continue

        # Remove inline single-line comments
        if '//' in line:
            line = line.split('//')[0].strip()

        # Print lines that are not comments
        if line and not annotation:
            print(line)
            output_file.write(process_line(line) + '\n')  # Add newline after each instruction
        if line and annotation:
            output_file.write(process_line(line) + f" // {line}" + '\n')  # Add newline after each instruction
            

ldr r0 101
ldr r1 011
lsl r0
lsl r3
mov r0 r0
00
lsr r0
mov r0 r0
00
lsr r0
mov r0 r0
00
lsr r0
mov r0 r0
00
lsl r1
mov r1 r1
01
lsr r1
mov r1 r1
01
lsr r1
mov r1 r1
01
lsr r1
mov r1 r1
01
ina 01
str r3 101
str r1 000
str r0 001
ina 00
mov r0 r2
10
ldr r0 000
orr r2
orr r1
lsl r2
lsl r2
lsl r1
lsl r1
ina 01
str r1 110
str r2 111
ldr r0 000
ldr r1 001
ina 00
cmp r1
bge 01110
cmp r1
bne 01111
ldr r2 101
ina 01
ldr r0 010
ina 00
and r2
ina 01
ldr r0 111
ina 00
add r2
mov r2 r0
00
add r4
ldr r1 011
ina 01
ldr r0 010
ina 00
and r1
ina 01
ldr r0 110
ina 00
add r1
mov r1 r0
00
add r5
mov r5 r0
00
str r0 011
ina 00
ldr r0 010
ldr r1 100
add r1
lsl r6
mov r1 r0
00
add r7
mov r4 r0
00
ldr r1 011
add r1
mov r6 r0
00
add r1
ina 01
str r1 100
ina 00
ldr r0 000
ldr r1 001
cmp r1
bne 10000
ldr r2 101
ina 01
ldr r0 010
ina 00
and r2
ina 01
ldr r0 111
ina 00
add r2
mov r2 r0
00
add r4
ldr r1 011
ina 01
ldr r0 010
ina 00
and r1
ina 01
ldr r0 110
ina 00
add r1
mov r1 r0
00
add r5
ldr r0 001
ina 01
ldr r1

In [2]:
# # Test harness
# with open('assembled.txt', 'r') as file1, open('cse141l_assembler_tests_key.txt', 'r') as file2:
#     for line_num, (line1, line2) in enumerate(zip(file1, file2), start=1):
#         if line1.strip() != line2.strip():
#             print(f"Difference at line {line_num}:")
#             print(f"Actual: {line1.strip()}")
#             print(f"Expected: {line2.strip()}")
#             print()